In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
import pandas as pd
import random
import time
from datetime import datetime, timedelta
import logging
import csv

logging.config.fileConfig('logger.config')

logger = logging
warn = logger.warning

In [2]:
user_agent_list = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

In [3]:
def multiplepage(url,user_agent_list):
    # Create a pool of proxies 

    try:
        
        user = random.choice(user_agent_list)
        logger.info("user agent is %s",user)
        headers={"accept-language": "en-US,en;q=0.9",
         "accept-encoding": "gzip, deflate, br",
         "User-Agent":user,
         "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"}
        
        # Send an HTTP request to the URL
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the HTML content of the response
        soup = BeautifulSoup(response.text, 'html.parser')

        

        return soup
    except requests.RequestException as e:
        print("Error fetching product data:", e)
        return None

# Player info extraction

In [5]:
url = "https://www.espncricinfo.com/series/icc-world-twenty20-2009-335113/squads"

In [6]:
squads_html = multiplepage(url,user_agent_list)

In [7]:
squads_json = (squads_html.find('script', {'id': '__NEXT_DATA__'})).contents[0]

In [8]:
decoded_data = json.loads(squads_json)

In [22]:
temp_list = []

In [17]:
temp_list.extend(decoded_data['props']['appPageProps']['data']['content']['squads'])

In [23]:
with open('../json/seasons.json','r') as sf:
    seasons = json.load(sf)
    for season in seasons:
        url = season['link']+"/squads"
        print(url)
        squads_html = multiplepage(url,user_agent_list)
        squads_json = (squads_html.find('script', {'id': '__NEXT_DATA__'})).contents[0]
        decoded_data = json.loads(squads_json)
        temp_list.extend(decoded_data['props']['appPageProps']['data']['content']['squads'])
        
        time.sleep(5)  # import time

https://www.espncricinfo.com/series/icc-world-twenty20-2009-335113/squads
https://www.espncricinfo.com/series/icc-world-twenty20-2010-412671/squads
https://www.espncricinfo.com//series/icc-world-twenty20-2012-13-531597/squads
https://www.espncricinfo.com//series/world-t20-2013-14-628368/squads
https://www.espncricinfo.com//series/world-t20-2015-16-901359/squads
https://www.espncricinfo.com//series/icc-men-s-t20-world-cup-2021-22-1267897/squads
https://www.espncricinfo.com//series/icc-men-s-t20-world-cup-2022-23-1298134/squads
https://www.espncricinfo.com//series/icc-men-s-t20-world-cup-2024-1411166/squads


In [24]:
with open('../json/all_sqads_raw.json','w') as rf:
    json.dump(temp_list,rf)

In [5]:
with open('../json/all_sqads_raw.json','r') as rf:
    temp_list = json.load(rf)

In [6]:
squad_urls = []

for i in temp_list:
    squad_objectid = i['squad']['objectId']
    seriesSlug = i['squad']['seriesSlug']
    seriesObjectId = i['squad']['seriesObjectId']
    teamSlug = i['squad']['slug']
    out = f"https://www.espncricinfo.com/series/{seriesSlug}-{seriesObjectId}/{teamSlug}-{squad_objectid}/series-squads"
    squad_urls.append(out)
    

In [7]:
len(squad_urls)
squad_urls

120

# Players Details extractoin

In [8]:
url = 'https://www.espncricinfo.com/series/icc-world-twenty20-2009-335113/australia-squad-402963/series-squads'

team_html = multiplepage(url,user_agent_list)
team_json = (team_html.find('script', {'id': '__NEXT_DATA__'})).contents[0]
decoded_data = json.loads(team_json)

In [12]:
players_list = []

for url in squad_urls:
    team_html = multiplepage(url,user_agent_list)
    team_json = (team_html.find('script', {'id': '__NEXT_DATA__'})).contents[0]
    decoded_data = json.loads(team_json)

    players_list.append(decoded_data['props']['appPageProps']['data']['content']['squadDetails']['players'])
    
    time.sleep(10)

In [15]:
temp = []
for x in players_list:
    temp.extend(x)


In [19]:
with open("../json/players.json",'w') as pf:
    json.dump(temp, pf)

# season, matches data extraction


In [13]:
with open('../../json/series.json','w') as f:
    json.dump(decoded_data,f)

In [25]:
table_json = decoded_data['props']['appPageProps']['data']['data']['content']['tables'][0]['rows']

In [30]:
data_dict = []
c = 1
for i in table_json:
    id = f"0{c}"
    title = i['items'][0]['value']
    link = i['items'][0]['link']
    year = i['items'][1]['value']
    winner = i['items'][2]['value']
    
    data_dict.append({'id':id,'title':title,'link':link,'year':year,'winner':winner})
    c+=1

In [31]:
with open('../../json/raw/data.json','w') as f:
    json.dump(data_dict,f)

In [38]:
matches_2007 = multiplepage(f"https://www.espncricinfo.com/{data_dict[0]['link']}",user_agent_list)
decoded_data = (matches_2007.find('script', {'id': '__NEXT_DATA__'})).contents[0]
matches_2007_json = json.loads(decoded_data)
with open('../../json/series/2007.json','w') as f:
    json.dump(matches_2007_json['props']['appPageProps']['data'],f)

In [18]:
with open('../../json/series/2007.json','r') as f:
    matches_2007_json = json.load(f)

In [19]:
series_data = matches_2007_json['series']
matches_data = matches_2007_json['content']['matches']

In [20]:
# Season details

series_id = series_data['id']
season = series_data['season']
year = series_data['year']
location = series_data['description']
start_date = (datetime.fromisoformat(series_data['startDate'])).date().isoformat()
end_date =  (datetime.fromisoformat(series_data['endDate'])).date().isoformat()
total_teams = series_data['totalSquads']


In [21]:
series_id,year,location,start_date,end_date,total_teams


(3519, 2008, 'in India', '2008-04-18', '2008-06-01', 8)

In [13]:
# Season matchs detail
i = matches_data[0]

season_id = i['series']['id']
match_id = i['id']
slug = i['slug']
title = i['title']
time_of_day = i['floodlit']
date = (datetime.fromisoformat(i['startDate'])).date().isoformat()
time = ((datetime.fromisoformat(i['startTime']))+timedelta(hours=5,minutes=30)).time().isoformat()
status = i['status']
status_desc = i['statusText']
winner_team_id = i['winnerTeamId']
toss_winner_team_id = i['tossWinnerTeamId']
toss_choice = i['tossWinnerChoice']
ground_id = i['ground']['id']
team_1_id = i['teams'][0]['team']['id']
team_2_id = i['teams'][1]['team']['id']
team_1_score = i['teams'][0]['score']
team_1_scoreInfo = i['teams'][0]['scoreInfo']
team_2_score = i['teams'][1]['score']
team_2_scoreInfo = i['teams'][1]['scoreInfo']





    

In [ ]:
season_id, match_id, slug, title, time_of_day, date, time, ground_id

In [ ]:
# ground details

ground = i['ground']
ground_id = ground['id']
ground_name = ground['name']
ground_slug = ground['slug']
ground_img = ground['image']['url']
town_id = ground['town']['id']
ground_capacity = ground['capacity']


In [18]:
# town details

town = ground['town']
town_id = town['id']
town_name = town['name']
country_id = ground['country']['id']


In [17]:
# Country details

country_id = ground['country']['id']
country_name = ground['country']['name']
country_abbrev = ground['country']['abbreviation']

In [20]:
# teams details

teams = i['teams']





In [5]:
mat_data = []
for i in matches_data:
    season_id = i['series']['id']
    match_id = i['id']
    slug = i['slug']
    title = i['title']
    time_of_day = i['floodlit']
    date = (datetime.fromisoformat(i['startDate'])).date().isoformat()
    time = ((datetime.fromisoformat(i['startTime']))+timedelta(hours=5,minutes=30)).time().isoformat()
    status = i['status']
    status_desc = i['statusText']
    winner_team_id = i['winnerTeamId']
    toss_winner_team_id = i['tossWinnerTeamId']
    toss_choice = i['tossWinnerChoice']
    ground_id = i['ground']['id']
    team_1_id = i['teams'][0]['team']['id']
    team_2_id = i['teams'][1]['team']['id']
    team_1_score = i['teams'][0]['score']
    team_1_scoreInfo = i['teams'][0]['scoreInfo']
    team_2_score = i['teams'][1]['score']
    team_2_scoreInfo = i['teams'][1]['scoreInfo']
    
    dict = {
        'season_id' : season_id,
        'match_id' : match_id,
        'slug' : slug,
        'title' : title,
        'time_of_day': time_of_day,  
        'date' : date,  
        'time' : time,  
        'status' : status,  
        'status_description' : status_desc,  
        'winner_team_id' : winner_team_id,  
        'toss_winner_team_id' : toss_winner_team_id,  
        'toss_choice' : toss_choice,  
        'ground_id' : ground_id,  
        'team_1' : {'id' : team_1_id, 'score' : team_1_score ,'score_info' : team_1_scoreInfo},  
        'team_2' : {'id' : team_2_id, 'score' : team_2_score ,'score_info' : team_2_scoreInfo}  # Additional layer for nested data
    }
    
    mat_data.append(dict)



In [54]:
with open('temp.json','w') as fl:
    json.dump(mat_data,fl)

# Insert into sql

In [6]:
import mysql.connector

# Establish connection to MySQL
try:
    # Replace 'username', 'password', and 'host' with your MySQL credentials
    connection = mysql.connector.connect(
        user='root',
        password='123456789',
        host='localhost',
        database = 'ipl',
        auth_plugin='mysql_native_password'
    )

    if connection.is_connected():
        print("Connected to MySQL")

        # Cursor to execute SQL queries
        cursor = connection.cursor()


except mysql.connector.Error as e:
    print("Error connecting to MySQL:", e)




Connected to MySQL


In [22]:
# Execute the INSERT query with placeholders
cursor.execute("INSERT INTO season VALUES (%s,%s,%s,%s,%s,%s)",(series_id,year,location,start_date,end_date,total_teams))

# Commit the transaction
connection.commit()
print("Data inserted successfully")

Data inserted successfully


In [ ]:
cursor.execute('''CREATE TABLE match (
      id integer PRIMARY KEY,
      season_id integer,
      slug varchar(255),
      title varchar(255),
      time_of_day varchar(255),
      date date,
      time timestamp,
      status varchar(255),
      status_desc varchar(255),
      winner_team_id integer,
      toss_winner_team_id integer,
      toss_choice integer,
      ground_id integer,
      team_1_id integer,
      team_2_id integer,
      team_1_score varchar(255),
      team_1_score_info varchar(255),
      team_2_score varchar(255),
      team_2_score_info varchar(255)
    )''')

In [8]:
for i in matches_data:
    season_id = i['series']['id']
    match_id = i['id']
    slug = i['slug']
    title = i['title']
    time_of_day = i['floodlit']
    date = (datetime.fromisoformat(i['startDate'])).date().isoformat()
    time = ((datetime.fromisoformat(i['startTime']))+timedelta(hours=5,minutes=30)).time().isoformat()
    status = i['status']
    status_desc = i['statusText']
    winner_team_id = i['winnerTeamId']
    toss_winner_team_id = i['tossWinnerTeamId']
    toss_choice = i['tossWinnerChoice']
    ground_id = i['ground']['id']
    team_1_id = i['teams'][0]['team']['id']
    team_2_id = i['teams'][1]['team']['id']
    team_1_score = i['teams'][0]['score']
    team_1_score_info = i['teams'][0]['scoreInfo']
    team_2_score = i['teams'][1]['score']
    team_2_score_info = i['teams'][1]['scoreInfo']
    
    cursor.execute("INSERT INTO ipl.match VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",
                   (match_id, season_id, slug, title, time_of_day, date, time, status, status_desc, winner_team_id, toss_winner_team_id, toss_choice, ground_id, team_1_id, team_2_id, team_1_score, team_1_score_info, team_2_score, team_2_score_info))

# Commit the transaction
connection.commit()
print("Data inserted successfully")
    

Data inserted successfully


In [13]:
# Execute SQL query to show all available databases
cursor.execute("select * from ipl.match")

# Fetch all rows
databases = cursor.fetchall()

# Print the list of databases
for i in databases:
    print(i)

[(17830, 3519, 'royal-challengers-bangalore-vs-kolkata-knight-riders-1st-match', '1st match', 'night', datetime.date(2008, 4, 18), datetime.timedelta(seconds=72000), 'RESULT', 'KKR won by 140 runs', 4341, 4340, 2, 683, 4341, 4340, '222/3', None, '82', '15.1/20 ov, T:223'), (17831, 3519, 'kings-xi-punjab-vs-chennai-super-kings-2nd-match', '2nd match', 'daynight', datetime.date(2008, 4, 19), datetime.timedelta(seconds=61200), 'RESULT', 'CSK won by 33 runs', 4343, 4343, 1, 1015, 4343, 4342, '240/5', None, '207/4', '20 ov, T:241'), (17832, 3519, 'delhi-daredevils-vs-rajasthan-royals-3rd-match', '3rd match', 'night', datetime.date(2008, 4, 19), datetime.timedelta(seconds=73800), 'RESULT', 'Daredevils won by 9 wickets (with 29 balls remaining)', 4344, 4345, 1, 333, 4345, 4344, '129/8', None, '132/1', '15.1/20 ov, T:130'), (17833, 3519, 'mumbai-indians-vs-royal-challengers-bangalore-5th-match', '5th match', 'night', datetime.date(2008, 4, 20), datetime.timedelta(seconds=72000), 'RESULT', 'RCB

In [ ]:

    # Close connection
if 'connection' in locals() and connection.is_connected():
    cursor.close()
    connection.close()
    print("MySQL connection closed")

In [14]:
cursor.execute('''create table if not exists `team` (
  `id` integer PRIMARY KEY,
  `name` varchar(255),
  `abbrev` varchar(255),
  `img_id` integer
);''')

In [15]:
# Execute SQL query to show all available tables
cursor.execute("show tables")

# Fetch all rows
tables = cursor.fetchall()

# Print the list of tables
for i in tables:
    print(i)

('match',)
('season',)
('team',)


In [2]:
import os
cwd = os.getcwd()
with open('../../json/pro/extract_from_seasons.json') as f:
    seasons_dict = json.load(f)